# [Reshaping and pivot tables](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html)
## Reshaping by pivoting DataFrame objects

![reshaping_pivot](img/reshaping_pivot.png)

Data is often stored in so-called “stacked” or “record” format:

```
In [1]: df
Out[1]: 
         date variable     value
0  2000-01-03        A  0.469112
1  2000-01-04        A -0.282863
2  2000-01-05        A -1.509059
3  2000-01-03        B -1.135632
4  2000-01-04        B  1.212112
5  2000-01-05        B -0.173215
6  2000-01-03        C  0.119209
7  2000-01-04        C -1.044236
8  2000-01-05        C -0.861849
9  2000-01-03        D -2.104569
10 2000-01-04        D -0.494929
11 2000-01-05        D  1.071804
```

For the curious here is how the above `DataFrame` was created:

In [1]:
import numpy as np
import pandas as pd

In [2]:
import pandas._testing as tm

def unpivot(frame):
    N, K = frame.shape
    data = {
        "value": frame.to_numpy().ravel("F"),
        "variable": np.asarray(frame.columns).repeat(N),
        "date": np.tile(np.asarray(frame.index), K),
    }
    return pd.DataFrame(data, columns=["date", "variable", "value"])


df = unpivot(tm.makeTimeDataFrame(3))
df

,date,variable,value
0,2000-01-03,A,-0.234966
1,2000-01-04,A,0.241004
2,2000-01-05,A,0.534023
3,2000-01-03,B,-0.404298
4,2000-01-04,B,-1.938244
5,2000-01-05,B,0.322305
6,2000-01-03,C,1.087294
7,2000-01-04,C,-2.554650
8,2000-01-05,C,0.087630
9,2000-01-03,D,0.705816


---

To select out everything for variable `A` we could do:

In [3]:
df[df["variable"] == "A"]

,date,variable,value
0,2000-01-03,A,-0.234966
1,2000-01-04,A,0.241004
2,2000-01-05,A,0.534023


But suppose we wish to do time series operations with the variables. A better representation would be where the `columns` are the unique variables and an `index` of dates identifies individual observations. To reshape the data into this form, we use the `DataFrame.pivot()` method (also implemented as a top level function `pivot()`):

In [4]:
df.pivot(index="date", columns="variable", values="value")

variable,A,B,C,D
date,,,,
2000-01-03,-0.234966,-0.404298,1.087294,0.705816
2000-01-04,0.241004,-1.938244,-2.554650,1.691101
2000-01-05,0.534023,0.322305,0.087630,0.593478


If the `values` argument is omitted, and the input `DataFrame` has more than one column of values which are not used as column or index inputs to `pivot`, then the resulting “pivoted” `DataFrame` will have `hierarchical columns` whose topmost level indicates the respective value column:

In [5]:
df["value2"] = df["value"] * 2
pivoted = df.pivot(index="date", columns="variable")
pivoted

value                                  value2            \
variable           A         B         C         D         A         B   
date                                                                     
2000-01-03 -0.234966 -0.404298  1.087294  0.705816 -0.469932 -0.808595   
2000-01-04  0.241004 -1.938244 -2.554650  1.691101  0.482008 -3.876488   
2000-01-05  0.534023  0.322305  0.087630  0.593478  1.068047  0.644610   

                                
variable           C         D  
date                            
2000-01-03  2.174589  1.411633  
2000-01-04 -5.109300  3.382203  
2000-01-05  0.175261  1.186955

You can then select subsets from the pivoted DataFrame:

In [6]:
pivoted["value2"]

variable,A,B,C,D
date,,,,
2000-01-03,-0.469932,-0.808595,2.174589,1.411633
2000-01-04,0.482008,-3.876488,-5.109300,3.382203
2000-01-05,1.068047,0.644610,0.175261,1.186955


Note that this returns a view on the underlying data in the case where the data are homogeneously-typed.

>Note
>
>`pivot()` will error with a `ValueError: Index contains duplicate entries, cannot reshape` if the index/column pair is not unique. In this case, consider using pivot_table() which is a generalization of pivot that can handle duplicate values for one index/column pair.

## Reshaping by stacking and unstacking

![reshaping_stack](img/reshaping_stack.png)

Closely related to the `pivot()` method are the related `stack()` and `unstack()` methods available on `Series` and `DataFrame`. These methods are designed to work together with `MultiIndex` objects (see the section on `hierarchical indexing`). Here are essentially what these methods do:

* `stack`: “pivot” a level of the (possibly hierarchical) column labels, returning a `DataFrame` with an index with a new inner-most level of row labels.

* `unstack`: (inverse operation of `stack`) “pivot” a level of the (possibly hierarchical) row index to the column axis, producing a reshaped `DataFrame` with a new inner-most level of column labels.

![reshaping_unstack](img/reshaping_unstack.png)

The clearest way to explain is by example. Let’s take a prior example data set from the hierarchical indexing section:

In [7]:
tuples = list(
    zip(
        *[
            ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
            ["one", "two", "one", "two", "one", "two", "one", "two"],
        ]
    )
)

index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])

df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=["A", "B"])

df2 = df[:4]

df2

A         B
first second                    
bar   one     1.896513 -0.395355
      two     1.563959 -1.455935
baz   one    -0.040790 -1.261485
      two    -0.018799 -0.526286

The `stack` function “compresses” a level in the `DataFrame`’s columns to produce either:

* A `Series`, in the case of a simple column Index.

* A `DataFrame`, in the case of a `MultiIndex` in the columns.

If the columns have a `MultiIndex`, you can choose which level to stack. The stacked level becomes the new lowest level in a `MultiIndex` on the columns:

In [8]:
stacked = df2.stack()
stacked

first  second   
bar    one     A    1.896513
               B   -0.395355
       two     A    1.563959
               B   -1.455935
baz    one     A   -0.040790
               B   -1.261485
       two     A   -0.018799
               B   -0.526286
dtype: float64

With a “stacked” `DataFrame` or `Series` (having a `MultiIndex` as the `index`), the inverse operation of `stack` is `unstack`, which by default unstacks the **last level**:

In [9]:
stacked.unstack()

A         B
first second                    
bar   one     1.896513 -0.395355
      two     1.563959 -1.455935
baz   one    -0.040790 -1.261485
      two    -0.018799 -0.526286

In [10]:
stacked.unstack(1)

second        one       two
first                      
bar   A  1.896513  1.563959
      B -0.395355 -1.455935
baz   A -0.040790 -0.018799
      B -1.261485 -0.526286

In [11]:
stacked.unstack(0)

first          bar       baz
second                      
one    A  1.896513 -0.040790
       B -0.395355 -1.261485
two    A  1.563959 -0.018799
       B -1.455935 -0.526286

![reshaping_unstack_1](img/reshaping_unstack_1.png)

If the indexes have names, you can use the level names instead of specifying the level numbers:

In [12]:
stacked.unstack("second")

second        one       two
first                      
bar   A  1.896513  1.563959
      B -0.395355 -1.455935
baz   A -0.040790 -0.018799
      B -1.261485 -0.526286

![reshaping_unstack_0](img/reshaping_unstack_0.png)

Notice that the `stack` and `unstack` methods implicitly sort the index levels involved. Hence a call to `stack` and then `unstack`, or vice versa, will result in a sorted copy of the original `DataFrame` or `Series`:

In [13]:
index = pd.MultiIndex.from_product([[2, 1], ["a", "b"]])
df = pd.DataFrame(np.random.randn(4), index=index, columns=["A"])
df

A
2 a -0.749743
  b  0.976038
1 a  1.013259
  b -0.360218

In [14]:
all(df.unstack().stack() == df.sort_index())

True

The above code will raise a TypeError if the call to sort_index is removed.

### Multiple levels
You may also stack or unstack more than one level at a time by passing a list of levels, in which case the end result is as if each level in the list were processed individually.

In [15]:
columns = pd.MultiIndex.from_tuples(
    [
        ("A", "cat", "long"),
        ("B", "cat", "long"),
        ("A", "dog", "short"),
        ("B", "dog", "short"),
    ],
    names=["exp", "animal", "hair_length"],
)

df = pd.DataFrame(np.random.randn(4, 4), columns=columns)
df

exp,A,B,A,B
animal,cat,cat,dog,dog
hair_length,long,long,short,short
0,1.595383,0.113616,1.086750,0.157409
1,-0.932750,-0.047743,0.387251,0.417135
2,0.161734,0.674086,0.877427,-0.030735
3,1.203758,-0.769334,1.160414,-1.711781


In [16]:
df.stack(level=["animal", "hair_length"])

exp                          A         B
  animal hair_length                    
0 cat    long         1.595383  0.113616
  dog    short        1.086750  0.157409
1 cat    long        -0.932750 -0.047743
  dog    short        0.387251  0.417135
2 cat    long         0.161734  0.674086
  dog    short        0.877427 -0.030735
3 cat    long         1.203758 -0.769334
  dog    short        1.160414 -1.711781

The list of levels can contain either level names or level numbers (but not a mixture of the two).

In [17]:
# df.stack(level=['animal', 'hair_length'])
# from above is equivalent to:
df.stack(level=[1, 2])

exp                          A         B
  animal hair_length                    
0 cat    long         1.595383  0.113616
  dog    short        1.086750  0.157409
1 cat    long        -0.932750 -0.047743
  dog    short        0.387251  0.417135
2 cat    long         0.161734  0.674086
  dog    short        0.877427 -0.030735
3 cat    long         1.203758 -0.769334
  dog    short        1.160414 -1.711781

### Missing data
These functions are intelligent about handling missing data and do not expect each subgroup within the hierarchical index to have the same set of labels. They also can handle the index being unsorted (but you can make it sorted by calling sort_index, of course). Here is a more complex example:



In [18]:
columns = pd.MultiIndex.from_tuples(
    [
        ("A", "cat"),
        ("B", "dog"),
        ("B", "cat"),
        ("A", "dog"),
    ],
    names=["exp", "animal"],
)

index = pd.MultiIndex.from_product(
    [("bar", "baz", "foo", "qux"), ("one", "two")], names=["first", "second"]
)

df = pd.DataFrame(np.random.randn(8, 4), index=index, columns=columns)
df2 = df.iloc[[0, 1, 2, 4, 5, 7]]
df2

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     1.573537 -0.944087  1.235844  1.453907
      two     0.283052  0.649462  0.104934 -0.238876
baz   one    -0.427063  0.164644  0.259161  0.618645
foo   one    -0.281813 -0.687820 -0.985681  1.397552
      two     0.273285 -0.535123  1.896377  1.236406
qux   two    -1.432166 -0.136347 -0.100723 -0.843688

As mentioned above, stack can be called with a level argument to select which level in the columns to stack:

In [19]:
df2.stack("exp")

animal                 cat       dog
first second exp                    
bar   one    A    1.573537  1.453907
             B    1.235844 -0.944087
      two    A    0.283052 -0.238876
             B    0.104934  0.649462
baz   one    A   -0.427063  0.618645
             B    0.259161  0.164644
foo   one    A   -0.281813  1.397552
             B   -0.985681 -0.687820
      two    A    0.273285  1.236406
             B    1.896377 -0.535123
qux   two    A   -1.432166 -0.843688
             B   -0.100723 -0.136347

In [20]:
df2.stack("animal")

exp                         A         B
first second animal                    
bar   one    cat     1.573537  1.235844
             dog     1.453907 -0.944087
      two    cat     0.283052  0.104934
             dog    -0.238876  0.649462
baz   one    cat    -0.427063  0.259161
             dog     0.618645  0.164644
foo   one    cat    -0.281813 -0.985681
             dog     1.397552 -0.687820
      two    cat     0.273285  1.896377
             dog     1.236406 -0.535123
qux   two    cat    -1.432166 -0.100723
             dog    -0.843688 -0.136347

Unstacking can result in missing values if subgroups do not have the same set of labels. By default, missing values will be replaced with the default fill value for that data type, NaN for float, NaT for datetimelike, etc. For integer types, by default data will converted to float and missing values will be set to NaN.

In [21]:
df3 = df.iloc[[0, 1, 4, 7], [1, 2]]
df3

exp                  B          
animal             dog       cat
first second                    
bar   one    -0.944087  1.235844
      two     0.649462  0.104934
foo   one    -0.687820 -0.985681
qux   two    -0.136347 -0.100723

In [22]:
df3.unstack()

exp            B                              
animal       dog                 cat          
second       one       two       one       two
first                                         
bar    -0.944087  0.649462  1.235844  0.104934
foo    -0.687820       NaN -0.985681       NaN
qux          NaN -0.136347       NaN -0.100723

Alternatively, unstack takes an optional fill_value argument, for specifying the value of missing data.

In [23]:
df3.unstack(fill_value=-1e9)

exp                B                                          
animal           dog                         cat              
second           one           two           one           two
first                                                         
bar    -9.440873e-01  6.494615e-01  1.235844e+00  1.049336e-01
foo    -6.878197e-01 -1.000000e+09 -9.856806e-01 -1.000000e+09
qux    -1.000000e+09 -1.363468e-01 -1.000000e+09 -1.007233e-01

### With a MultiIndex
Unstacking when the columns are a MultiIndex is also careful about doing the right thing:

In [24]:
df[:3].unstack(0)

exp            A                   B                                       A  \
animal       cat                 dog                 cat                 dog   
first        bar       baz       bar       baz       bar       baz       bar   
second                                                                         
one     1.573537 -0.427063 -0.944087  0.164644  1.235844  0.259161  1.453907   
two     0.283052       NaN  0.649462       NaN  0.104934       NaN -0.238876   

exp               
animal            
first        baz  
second            
one     0.618645  
two          NaN

In [25]:
df2.unstack(1)

exp            A                   B                                       A  \
animal       cat                 dog                 cat                 dog   
second       one       two       one       two       one       two       one   
first                                                                          
bar     1.573537  0.283052 -0.944087  0.649462  1.235844  0.104934  1.453907   
baz    -0.427063       NaN  0.164644       NaN  0.259161       NaN  0.618645   
foo    -0.281813  0.273285 -0.687820 -0.535123 -0.985681  1.896377  1.397552   
qux          NaN -1.432166       NaN -0.136347       NaN -0.100723       NaN   

exp               
animal            
second       two  
first             
bar    -0.238876  
baz          NaN  
foo     1.236406  
qux    -0.843688

## Reshaping by melt

![reshaping_melt](img/reshaping_melt.png)

The top-level `melt()` function and the corresponding `DataFrame.melt()` are useful to massage a `DataFrame` into a format where one or more columns are identifier variables, while all other columns, considered measured variables, are “unpivoted” to the row axis, leaving just two non-identifier columns, “variable” and “value”. The names of those columns can be customized by supplying the `var_name` and `value_name` parameters.

For instance,

In [26]:
cheese = pd.DataFrame(
    {
        "first": ["John", "Mary"],
        "last": ["Doe", "Bo"],
        "height": [5.5, 6.0],
        "weight": [130, 150],
    }
)
cheese

,first,last,height,weight
0,John,Doe,5.5,130
1,Mary,Bo,6.0,150


In [27]:
cheese.melt(id_vars=["first", "last"])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [28]:
cheese.melt(id_vars=["first", "last"], var_name="quantity")

,first,last,quantity,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


When transforming a DataFrame using `melt()`, the index will be ignored. The original index values can be kept around by setting the `ignore_index` parameter to `False` (default is `True`). This will however duplicate them.

*New in version 1.1.0*.

In [29]:
index = pd.MultiIndex.from_tuples([("person", "A"), ("person", "B")])

cheese = pd.DataFrame(
    {
        "first": ["John", "Mary"],
        "last": ["Doe", "Bo"],
        "height": [5.5, 6.0],
        "weight": [130, 150],
    },
    index=index,
)
cheese

first last  height  weight
person A  John  Doe     5.5     130
       B  Mary   Bo     6.0     150

In [30]:
cheese.melt(id_vars=["first", "last"])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [31]:
cheese.melt(id_vars=["first", "last"], ignore_index=False)

first last variable  value
person A  John  Doe   height    5.5
       B  Mary   Bo   height    6.0
       A  John  Doe   weight  130.0
       B  Mary   Bo   weight  150.0

Another way to transform is to use the `wide_to_long()` panel data convenience function. It is less flexible than `melt()`, but more user-friendly.

In [32]:
dft = pd.DataFrame(
    {
        "A1970": {0: "a", 1: "b", 2: "c"},
        "A1980": {0: "d", 1: "e", 2: "f"},
        "B1970": {0: 2.5, 1: 1.2, 2: 0.7},
        "B1980": {0: 3.2, 1: 1.3, 2: 0.1},
        "X": dict(zip(range(3), np.random.randn(3))),
    }
)
dft["id"] = dft.index
dft

,A1970,A1980,B1970,B1980,X,id
0,a,d,2.5,3.2,0.631308,0
1,b,e,1.2,1.3,0.682625,1
2,c,f,0.7,0.1,-0.179587,2


In [33]:
pd.wide_to_long(dft, ["A", "B"], i="id", j="year")

,,X,A,B
id,year,,,
0,1970,0.631308,a,2.5
1,1970,0.682625,b,1.2
2,1970,-0.179587,c,0.7
0,1980,0.631308,d,3.2
1,1980,0.682625,e,1.3
2,1980,-0.179587,f,0.1


## Combining with stats and GroupBy
It should be no shock that combining `pivot` / `stack` / `unstack` with GroupBy and the basic Series and DataFrame statistical functions can produce some very expressive and fast data manipulations.

In [34]:
df

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     1.573537 -0.944087  1.235844  1.453907
      two     0.283052  0.649462  0.104934 -0.238876
baz   one    -0.427063  0.164644  0.259161  0.618645
      two     0.160963  2.199277  0.428027  0.129297
foo   one    -0.281813 -0.687820 -0.985681  1.397552
      two     0.273285 -0.535123  1.896377  1.236406
qux   one    -0.587985  0.067903 -0.818199 -0.747226
      two    -1.432166 -0.136347 -0.100723 -0.843688

In [35]:
df.stack().mean(1).unstack()

animal             cat       dog
first second                    
bar   one     1.404691  0.254910
      two     0.193993  0.205293
baz   one    -0.083951  0.391644
      two     0.294495  1.164287
foo   one    -0.633747  0.354866
      two     1.084831  0.350642
qux   one    -0.703092 -0.339661
      two    -0.766445 -0.490017

In [36]:
# same result, another way
In [56]: df.groupby(level=1, axis=1).mean()

animal             cat       dog
first second                    
bar   one     1.404691  0.254910
      two     0.193993  0.205293
baz   one    -0.083951  0.391644
      two     0.294495  1.164287
foo   one    -0.633747  0.354866
      two     1.084831  0.350642
qux   one    -0.703092 -0.339661
      two    -0.766445 -0.490017

In [37]:
df.stack().groupby(level=1).mean()

exp,A,B
second,,
one,0.374944,-0.213529
two,-0.053966,0.563236


In [38]:
df.mean().unstack(0)

exp,A,B
animal,,
cat,-0.054774,0.252468
dog,0.375752,0.097239


## Pivot tables
While `pivot()` provides general purpose pivoting with various data types (strings, numerics, etc.), pandas also provides `pivot_table()` for pivoting with aggregation of numeric data.

The function `pivot_table()` can be used to create spreadsheet-style pivot tables. See the `cookbook` for some advanced strategies.

It takes a number of arguments:

* `data`: a DataFrame object.

* `values`: a column or a list of columns to aggregate.

* `index`: a column, Grouper, array which has the same length as data, or list of them. Keys to group by on the pivot table index. If an array is passed, it is being used as the same manner as column values.

* `columns`: a column, Grouper, array which has the same length as data, or list of them. Keys to group by on the pivot table column. If an array is passed, it is being used as the same manner as column values.

* `aggfunc`: function to use for aggregation, defaulting to numpy.mean.

Consider a data set like this:



In [39]:
import datetime

df = pd.DataFrame(
    {
        "A": ["one", "one", "two", "three"] * 6,
        "B": ["A", "B", "C"] * 8,
        "C": ["foo", "foo", "foo", "bar", "bar", "bar"] * 4,
        "D": np.random.randn(24),
        "E": np.random.randn(24),
        "F": [datetime.datetime(2013, i, 1) for i in range(1, 13)]
        + [datetime.datetime(2013, i, 15) for i in range(1, 13)],
    }
)
df

,A,B,C,D,E,F
0,one,A,foo,0.079701,-1.180231,2013-01-01
1,one,B,foo,0.762483,1.378266,2013-02-01
2,two,C,foo,0.352488,-1.495306,2013-03-01
3,three,A,bar,-1.443830,0.488386,2013-04-01
4,one,B,bar,-0.836892,-0.917329,2013-05-01
5,one,C,bar,-0.192374,-1.437893,2013-06-01
6,two,A,foo,-1.241767,0.571475,2013-07-01
7,three,B,foo,2.225198,-0.174675,2013-08-01
8,one,C,foo,-0.105443,-0.310389,2013-09-01
9,one,A,bar,0.217556,0.870856,2013-10-01


We can produce pivot tables from this data very easily:

In [40]:
 pd.pivot_table(df, values="D", index=["A", "B"], columns=["C"])

C             bar       foo
A     B                    
one   A  0.079337  0.648510
      B -0.169639 -0.915210
      C -0.671010 -0.431653
three A -1.323364       NaN
      B       NaN  1.282900
      C  0.213155       NaN
two   A       NaN -0.703783
      B  0.680394       NaN
      C       NaN  0.994588

In [41]:
pd.pivot_table(df, values="D", index=["B"], columns=["A", "C"], aggfunc=np.sum)

A       one               three               two          
C       bar       foo       bar     foo       bar       foo
B                                                          
A  0.158674  1.297021 -2.646728     NaN       NaN -1.407565
B -0.339278 -1.830420       NaN  2.5658  1.360787       NaN
C -1.342019 -0.863305  0.426309     NaN       NaN  1.989175

In [42]:
pd.pivot_table(
    df, values=["D", "E"],
    index=["B"],
    columns=["A", "C"],
    aggfunc=np.sum,
)

D                                                         E  \
A       one               three               two                 one   
C       bar       foo       bar     foo       bar       foo       bar   
B                                                                       
A  0.158674  1.297021 -2.646728     NaN       NaN -1.407565 -2.234982   
B -0.339278 -1.830420       NaN  2.5658  1.360787       NaN -0.432870   
C -1.342019 -0.863305  0.426309     NaN       NaN  1.989175 -0.652680   

                                                     
A               three                 two            
C       foo       bar       foo       bar       foo  
B                                                    
A -0.108025 -1.670121       NaN       NaN  0.626571  
B -0.070654       NaN  0.032926  0.318276       NaN  
C  0.317015  2.724579       NaN       NaN -1.438612

The result object is a `DataFrame` having potentially hierarchical indexes on the rows and columns. If the `values` column name is not given, the pivot table will include all of the data that can be aggregated in an additional level of hierarchy in the columns:

In [43]:
pd.pivot_table(df, index=["A", "B"], columns=["C"])

D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A  0.079337  0.648510 -1.117491 -0.054012
      B -0.169639 -0.915210 -0.216435 -0.035327
      C -0.671010 -0.431653 -0.326340  0.158508
three A -1.323364       NaN -0.835060       NaN
      B       NaN  1.282900       NaN  0.016463
      C  0.213155       NaN  1.362290       NaN
two   A       NaN -0.703783       NaN  0.313285
      B  0.680394       NaN  0.159138       NaN
      C       NaN  0.994588       NaN -0.719306

Also, you can use Grouper for index and columns keywords. For detail of Grouper, see Grouping with a Grouper specification.



In [44]:
pd.pivot_table(df, values="D", index=pd.Grouper(freq="M", key="F"), columns="C")

C,bar,foo
F,,
2013-01-31,NaN,0.648510
2013-02-28,NaN,-0.915210
2013-03-31,NaN,0.994588
2013-04-30,-1.323364,NaN
2013-05-31,-0.169639,NaN
2013-06-30,-0.671010,NaN
2013-07-31,NaN,-0.703783
2013-08-31,NaN,1.282900
2013-09-30,NaN,-0.431653


You can render a nice output of the table omitting the missing values by calling `to_string` if you wish:

In [45]:
table = pd.pivot_table(df, index=["A", "B"], columns=["C"])
print(table.to_string(na_rep=""))

                D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A  0.079337  0.648510 -1.117491 -0.054012
      B -0.169639 -0.915210 -0.216435 -0.035327
      C -0.671010 -0.431653 -0.326340  0.158508
three A -1.323364           -0.835060          
      B            1.282900            0.016463
      C  0.213155            1.362290          
two   A           -0.703783            0.313285
      B  0.680394            0.159138          
      C            0.994588           -0.719306


Note that `pivot_table` is also available as an instance method on DataFrame,
i.e. `DataFrame.pivot_table()`.

### dding margins
If you pass `margins=True` to `pivot_table`, special `All` columns and rows will be added with partial group aggregates across the categories on the rows and columns:

In [46]:
df.pivot_table(index=["A", "B"], columns="C", margins=True, aggfunc=np.std)

D                             E                    
C             bar       foo       All       bar       foo       All
A     B                                                            
one   A  0.195471  0.804419  0.580016  2.811947  1.592714  1.964245
      B  0.943639  2.372616  1.535756  0.991214  1.999122  1.292516
      C  0.676893  0.461330  0.492714  1.571973  0.663120  1.024029
three A  0.170364       NaN  0.170364  1.871636       NaN  1.871636
      B       NaN  1.332611  1.332611       NaN  0.270310  0.270310
      C  0.292923       NaN  0.292923  1.290477       NaN  1.290477
two   A       NaN  0.760825  0.760825       NaN  0.365135  0.365135
      B  0.167512       NaN  0.167512  0.303303       NaN  0.303303
      C       NaN  0.908066  0.908066       NaN  1.097430  1.097430
All      0.771468  1.299048  1.037250  1.485263  0.936128  1.189834

## Cross tabulations
Use `crosstab()` to compute a cross-tabulation of two (or more) factors. By default `crosstab` computes a frequency table of the factors unless an array of values and an aggregation function are passed.

It takes a number of arguments

* `index`: array-like, values to group by in the rows.

* `columns`: array-like, values to group by in the columns.

* `values`: array-like, optional, array of values to aggregate according to the factors.

* `aggfunc`: function, optional, If no values array is passed, computes a frequency table.

* `rownames`: sequence, default `None`, must match number of row arrays passed.

* `colnames`: sequence, default `None`, if passed, must match number of column arrays passed.

* `margins`: boolean, default `False`, Add row/column margins (subtotals)

* `normalize`: boolean, {‘all’, ‘index’, ‘columns’}, or {0,1}, default `False`. Normalize by dividing all values by the sum of values.

Any `Series` passed will have their name attributes used unless row or column names for the cross-tabulation are specified

For example:

In [47]:
foo, bar, dull, shiny, one, two = "foo", "bar", "dull", "shiny", "one", "two"
a = np.array([foo, foo, bar, bar, foo, foo], dtype=object)
b = np.array([one, one, two, one, two, one], dtype=object)
c = np.array([dull, dull, shiny, dull, dull, shiny], dtype=object)
pd.crosstab(a, [b, c], rownames=["a"], colnames=["b", "c"])

b    one        two      
c   dull shiny dull shiny
a                        
bar    1     0    0     1
foo    2     1    1     0

If `crosstab` receives only two Series, it will provide a frequency table.

In [48]:
df = pd.DataFrame(
    {"A": [1, 2, 2, 2, 2], "B": [3, 3, 4, 4, 4], "C": [1, 1, np.nan, 1, 1]}
)
df

,A,B,C
0,1,3,1.0
1,2,3,1.0
2,2,4,NaN
3,2,4,1.0
4,2,4,1.0


In [49]:
pd.crosstab(df["A"], df["B"])

B,3,4
A,,
1,1,0
2,1,3


`crosstab` can also be implemented to Categorical data.

In [50]:
foo = pd.Categorical(["a", "b"], categories=["a", "b", "c"])
bar = pd.Categorical(["d", "e"], categories=["d", "e", "f"])
pd.crosstab(foo, bar)

col_0,d,e
row_0,,
a,1,0
b,0,1


If you want to include all of data categories even if the actual data does not contain any instances of a particular category, you should set `dropna=False`.

For example:

In [51]:
pd.crosstab(foo, bar, dropna=False)

col_0,d,e,f
row_0,,,
a,1,0,0
b,0,1,0
c,0,0,0


### Normalization
Frequency tables can also be normalized to show percentages rather than counts using the `normalize` argument:

In [52]:
pd.crosstab(df["A"], df["B"], normalize=True)

B,3,4
A,,
1,0.2,0.0
2,0.2,0.6


`normalize` can also normalize values within each row or within each column:

In [53]:
pd.crosstab(df["A"], df["B"], normalize="columns")

B,3,4
A,,
1,0.5,0.0
2,0.5,1.0


`crosstab` can also be passed a third `Series` and an aggregation function (`aggfunc`) that will be applied to the values of the third `Series` within each group defined by the first two `Series`:

In [54]:
pd.crosstab(df["A"], df["B"], values=df["C"], aggfunc=np.sum)

B,3,4
A,,
1,1.0,NaN
2,1.0,2.0


### Adding margins
Finally, one can also add margins or normalize this output.

In [55]:
pd.crosstab(
    df["A"], df["B"], values=df["C"], aggfunc=np.sum, normalize=True, margins=True
)

B,3,4,All
A,,,
1,0.25,0.0,0.25
2,0.25,0.5,0.75
All,0.50,0.5,1.00


## Tiling
The `cut()` function computes groupings for the values of the input array and is often used to transform continuous variables to discrete or categorical variables:

In [56]:
ages = np.array([10, 15, 13, 12, 23, 25, 28, 59, 60])
pd.cut(ages, bins=3)

[(9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (26.667, 43.333], (43.333, 60.0], (43.333, 60.0]]
Categories (3, interval[float64]): [(9.95, 26.667] < (26.667, 43.333] < (43.333, 60.0]]

If the `bins` keyword is an integer, then equal-width bins are formed. Alternatively we can specify custom bin-edges:

In [57]:
c = pd.cut(ages, bins=[0, 18, 35, 70])
c

[(0, 18], (0, 18], (0, 18], (0, 18], (18, 35], (18, 35], (18, 35], (35, 70], (35, 70]]
Categories (3, interval[int64]): [(0, 18] < (18, 35] < (35, 70]]

If the bins keyword is an IntervalIndex, then these will be used to bin the passed data.:

```python
pd.cut([25, 20, 50], bins=c.categories)
```

## Computing indicator / dummy variables
To convert a categorical variable into a “dummy” or “indicator” DataFrame, for example a column in a DataFrame (a Series) which has k distinct values, can derive a DataFrame containing k columns of 1s and 0s using get_dummies():

In [58]:
df = pd.DataFrame({"key": list("bbacab"), "data1": range(6)})
pd.get_dummies(df["key"])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


Sometimes it’s useful to prefix the column names, for example when merging the result with the original DataFrame:

In [59]:
dummies = pd.get_dummies(df["key"], prefix="key")
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [60]:
df[["data1"]].join(dummies)

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


This function is often used along with discretization functions like cut:

In [61]:
values = np.random.randn(10)
values

array([-0.71115167, -1.96200267, -0.41058706, -0.98456331,  0.64256822,
       -1.0160962 , -1.01577447,  1.14976434, -2.20554521,  0.49811064])

In [62]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,1,0
5,0,0,0,0,0
6,0,0,0,0,0
7,0,0,0,0,0
8,0,0,0,0,0
9,0,0,1,0,0


See also Series.str.get_dummies.

get_dummies() also accepts a DataFrame. By default all categorical variables (categorical in the statistical sense, those with object or categorical dtype) are encoded as dummy variables.

In [63]:
df = pd.DataFrame({"A": ["a", "b", "a"], "B": ["c", "c", "b"], "C": [1, 2, 3]})
pd.get_dummies(df)

,C,A_a,A_b,B_b,B_c
0,1,1,0,0,1
1,2,0,1,0,1
2,3,1,0,1,0


All non-object columns are included untouched in the output. You can control the columns that are encoded with the columns keyword.

In [64]:
pd.get_dummies(df, columns=["A"])

,B,C,A_a,A_b
0,c,1,1,0
1,c,2,0,1
2,b,3,1,0


Notice that the B column is still included in the output, it just hasn’t been encoded. You can drop B before calling get_dummies if you don’t want to include it in the output.

As with the Series version, you can pass values for the prefix and prefix_sep. By default the column name is used as the prefix, and ‘_’ as the prefix separator. You can specify prefix and prefix_sep in 3 ways:

* string: Use the same value for prefix or prefix_sep for each column to be encoded.

* list: Must be the same length as the number of columns being encoded.

* dict: Mapping column name to prefix.

In [65]:
simple = pd.get_dummies(df, prefix="new_prefix")
simple

,C,new_prefix_a,new_prefix_b,new_prefix_b,new_prefix_c
0,1,1,0,0,1
1,2,0,1,0,1
2,3,1,0,1,0


In [66]:
from_list = pd.get_dummies(df, prefix=["from_A", "from_B"])
from_list

,C,from_A_a,from_A_b,from_B_b,from_B_c
0,1,1,0,0,1
1,2,0,1,0,1
2,3,1,0,1,0


In [67]:
from_dict = pd.get_dummies(df, prefix={"B": "from_B", "A": "from_A"})
from_dict

,C,from_A_a,from_A_b,from_B_b,from_B_c
0,1,1,0,0,1
1,2,0,1,0,1
2,3,1,0,1,0


Sometimes it will be useful to only keep k-1 levels of a categorical variable to avoid collinearity when feeding the result to statistical models. You can switch to this mode by turn on drop_first.



In [68]:
s = pd.Series(list("abcaa"))
pd.get_dummies(s)

,a,b,c
0,1,0,0
1,0,1,0
2,0,0,1
3,1,0,0
4,1,0,0


In [69]:
pd.get_dummies(s, drop_first=True)

,b,c
0,0,0
1,1,0
2,0,1
3,0,0
4,0,0


When a column contains only one level, it will be omitted in the result.

In [70]:
df = pd.DataFrame({"A": list("aaaaa"), "B": list("ababc")})
pd.get_dummies(df)

,A_a,B_a,B_b,B_c
0,1,1,0,0
1,1,0,1,0
2,1,1,0,0
3,1,0,1,0
4,1,0,0,1


In [71]:
pd.get_dummies(df, drop_first=True)

,B_b,B_c
0,0,0
1,1,0
2,0,0
3,1,0
4,0,1


By default new columns will have np.uint8 dtype. To choose another dtype, use the dtype argument:

In [72]:
df = pd.DataFrame({"A": list("abc"), "B": [1.1, 2.2, 3.3]})
pd.get_dummies(df, dtype=bool).dtypes

B      float64
A_a       bool
A_b       bool
A_c       bool
dtype: object

## Factorizing values
To encode 1-d values as an enumerated type use factorize():

In [73]:
x = pd.Series(["A", "A", np.nan, "B", 3.14, np.inf])
x

0       A
1       A
2     NaN
3       B
4    3.14
5     inf
dtype: object

In [74]:
labels, uniques = pd.factorize(x)
labels

array([ 0,  0, -1,  1,  2,  3])

In [75]:
uniques

Index(['A', 'B', 3.14, inf], dtype='object')

Note that factorize is similar to numpy.unique, but differs in its handling of NaN:

>ote
>
>The following numpy.unique will fail under Python 3 with a TypeError because of an ordering bug. See also here.

In [76]:
x = pd.Series(['A', 'A', np.nan, 'B', 3.14, np.inf])
pd.factorize(x, sort=True)

(array([ 2,  2, -1,  3,  0,  1]), Index([3.14, inf, 'A', 'B'], dtype='object'))

In [77]:
import sys
try:
    np.unique(x, return_inverse=True)[::-1]    
except:
    print("Unexpected error:", sys.exc_info()[0])

Unexpected error: <class 'TypeError'>


>Note
>
>If you just want to handle one column as a categorical variable (like R’s factor), you can use df["cat_col"] = pd.Categorical(df["col"]) or df["cat_col"] = df["col"].astype("category"). For full docs on Categorical, see the Categorical introduction and the API documentation.

## Examples
In this section, we will review frequently asked questions and examples. The column names and relevant column values are named to correspond with how this DataFrame will be pivoted in the answers below.

In [78]:
np.random.seed([3, 1415])
n = 20
cols = np.array(["key", "row", "item", "col"])
df = cols + pd.DataFrame(
    (np.random.randint(5, size=(n, 4)) // [2, 1, 2, 1]).astype(str)
)

In [79]:
df.columns = cols
df = df.join(pd.DataFrame(np.random.rand(n, 2).round(2)).add_prefix("val"))
df

,key,row,item,col,val0,val1
0,key0,row3,item1,col3,0.81,0.04
1,key1,row2,item1,col2,0.44,0.07
2,key1,row0,item1,col0,0.77,0.01
3,key0,row4,item0,col2,0.15,0.59
4,key1,row0,item2,col1,0.81,0.64
5,key1,row2,item2,col4,0.13,0.88
6,key2,row4,item1,col3,0.88,0.39
7,key1,row4,item1,col1,0.10,0.07
8,key1,row0,item2,col4,0.65,0.02
9,key1,row2,item0,col2,0.35,0.61


### Pivoting with single aggregations
Suppose we wanted to pivot df such that the col values are columns, row values are the index, and the mean of val0 are the values? In particular, the resulting DataFrame should look like:

```
col   col0   col1   col2   col3  col4
row
row0  0.77  0.605    NaN  0.860  0.65
row2  0.13    NaN  0.395  0.500  0.25
row3   NaN  0.310    NaN  0.545   NaN
row4   NaN  0.100  0.395  0.760  0.24
```

This solution uses pivot_table(). Also note that aggfunc='mean' is the default. It is included here to be explicit.

In [80]:
df.pivot_table(values="val0", index="row", columns="col", aggfunc="mean")

col,col0,col1,col2,col3,col4
row,,,,,
row0,0.77,0.605,NaN,0.860,0.65
row2,0.13,NaN,0.395,0.500,0.25
row3,NaN,0.310,NaN,0.545,NaN
row4,NaN,0.100,0.395,0.760,0.24


Note that we can also replace the missing values by using the fill_value parameter.

In [81]:
df.pivot_table(
    values="val0",
    index="row",
    columns="col",
    aggfunc="mean",
    fill_value=0,
)

col,col0,col1,col2,col3,col4
row,,,,,
row0,0.77,0.605,0.000,0.860,0.65
row2,0.13,0.000,0.395,0.500,0.25
row3,0.00,0.310,0.000,0.545,0.00
row4,0.00,0.100,0.395,0.760,0.24


Also note that we can pass in other aggregation functions as well. For example, we can also pass in sum.

In [82]:
df.pivot_table(
    values="val0",
    index="row",
    columns="col",
    aggfunc="sum",
    fill_value=0,
)

col,col0,col1,col2,col3,col4
row,,,,,
row0,0.77,1.21,0.00,0.86,0.65
row2,0.13,0.00,0.79,0.50,0.50
row3,0.00,0.31,0.00,1.09,0.00
row4,0.00,0.10,0.79,1.52,0.24


Another aggregation we can do is calculate the frequency in which the columns and rows occur together a.k.a. “cross tabulation”. To do this, we can pass size to the aggfunc parameter.

In [83]:
df.pivot_table(index="row", columns="col", fill_value=0, aggfunc="size")

col,col0,col1,col2,col3,col4
row,,,,,
row0,1,2,0,1,1
row2,1,0,2,1,2
row3,0,1,0,2,0
row4,0,1,2,2,1


### Pivoting with multiple aggregations
We can also perform multiple aggregations. For example, to perform both a sum and mean, we can pass in a list to the aggfunc argument.

In [84]:
df.pivot_table(
    values="val0",
    index="row",
    columns="col",
    aggfunc=["mean", "sum"],
)

mean                              sum                        
col   col0   col1   col2   col3  col4  col0  col1  col2  col3  col4
row                                                                
row0  0.77  0.605    NaN  0.860  0.65  0.77  1.21   NaN  0.86  0.65
row2  0.13    NaN  0.395  0.500  0.25  0.13   NaN  0.79  0.50  0.50
row3   NaN  0.310    NaN  0.545   NaN   NaN  0.31   NaN  1.09   NaN
row4   NaN  0.100  0.395  0.760  0.24   NaN  0.10  0.79  1.52  0.24

Note to aggregate over multiple value columns, we can pass in a list to the values parameter.

In [85]:
df.pivot_table(
    values=["val0", "val1"],
    index="row",
    columns="col",
    aggfunc=["mean"],
)

mean                                                           
      val0                             val1                          
col   col0   col1   col2   col3  col4  col0   col1  col2   col3  col4
row                                                                  
row0  0.77  0.605    NaN  0.860  0.65  0.01  0.745   NaN  0.010  0.02
row2  0.13    NaN  0.395  0.500  0.25  0.45    NaN  0.34  0.440  0.79
row3   NaN  0.310    NaN  0.545   NaN   NaN  0.230   NaN  0.075   NaN
row4   NaN  0.100  0.395  0.760  0.24   NaN  0.070  0.42  0.300  0.46

Note to subdivide over multiple columns we can pass in a list to the columns parameter.

In [86]:
df.pivot_table(
    values=["val0"],
    index="row",
    columns=["item", "col"],
    aggfunc=["mean"],
)

mean                                                                   
      val0                                                                   
item item0             item1                         item2                   
col   col2  col3  col4  col0  col1  col2  col3  col4  col0   col1  col3  col4
row                                                                          
row0   NaN   NaN   NaN  0.77   NaN   NaN   NaN   NaN   NaN  0.605  0.86  0.65
row2  0.35   NaN  0.37   NaN   NaN  0.44   NaN   NaN  0.13    NaN  0.50  0.13
row3   NaN   NaN   NaN   NaN  0.31   NaN  0.81   NaN   NaN    NaN  0.28   NaN
row4  0.15  0.64   NaN   NaN  0.10  0.64  0.88  0.24   NaN    NaN   NaN   NaN

## Exploding a list-like column
New in version 0.25.0.

Sometimes the values in a column are list-like.

In [87]:
keys = ["panda1", "panda2", "panda3"]
values = [["eats", "shoots"], ["shoots", "leaves"], ["eats", "leaves"]]
df = pd.DataFrame({"keys": keys, "values": values})
df

,keys,values
0,panda1,"[eats, shoots]"
1,panda2,"[shoots, leaves]"
2,panda3,"[eats, leaves]"


We can ‘explode’ the values column, transforming each list-like to a separate row, by using explode(). This will replicate the index values from the original row:

In [88]:
df["values"].explode()

0      eats
0    shoots
1    shoots
1    leaves
2      eats
2    leaves
Name: values, dtype: object

You can also explode the column in the DataFrame.

In [89]:
df.explode("values")

,keys,values
0,panda1,eats
0,panda1,shoots
1,panda2,shoots
1,panda2,leaves
2,panda3,eats
2,panda3,leaves


Series.explode() will replace empty lists with np.nan and preserve scalar entries. The dtype of the resulting Series is always object.

In [90]:
s = pd.Series([[1, 2, 3], "foo", [], ["a", "b"]])
s

0    [1, 2, 3]
1          foo
2           []
3       [a, b]
dtype: object

In [91]:
s.explode()

0      1
0      2
0      3
1    foo
2    NaN
3      a
3      b
dtype: object

Here is a typical usecase. You have comma separated strings in a column and want to expand this.

In [92]:
df = pd.DataFrame([{"var1": "a,b,c", "var2": 1}, {"var1": "d,e,f", "var2": 2}])
df

,var1,var2
0,"a,b,c",1
1,"d,e,f",2


Creating a long form DataFrame is now straightforward using explode and chained operations

In [93]:
df.assign(var1=df.var1.str.split(",")).explode("var1")

,var1,var2
0,a,1
0,b,1
0,c,1
1,d,2
1,e,2
1,f,2
